In [19]:
# %% [markdown]
# # Chess Player Performance Analyzer - Complete Version
# 
# Comprehensive analysis tool for chess players including game duration analysis,
# temporal trends, and multiple export options.

# %% [markdown]
# ## 1. Setup and Imports

# %%
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import os
warnings.filterwarnings('ignore')

# Optional: Import chess.pgn if available
try:
    import chess.pgn
    PGN_AVAILABLE = True
except ImportError:
    PGN_AVAILABLE = False
    print("⚠️ python-chess not installed. PGN processing disabled.")

print("✅ Libraries imported successfully")

# %% [markdown]
# ## 2. Load and Process Data

# %%
def process_pgn_to_dataframe(pgn_file_path, num_games=None):
    """
    Process PGN file and convert to DataFrame format.
    """
    if not PGN_AVAILABLE:
        print("❌ PGN processing unavailable. Please install python-chess.")
        return pd.DataFrame()
    
    games = []
    with open(pgn_file_path) as pgn:
        for i, game in enumerate(iter(lambda: chess.pgn.read_game(pgn), None)):
            if num_games and i >= num_games:
                break
            
            try:
                headers = game.headers
                
                # Count moves
                move_count = 0
                node = game
                while node.variations:
                    node = node.variation(0)
                    move_count += 1
                
                # Full move count (divide by 2 for complete moves)
                full_move_count = (move_count + 1) // 2
                
                games.append({
                    'White': headers.get('White', 'Unknown'),
                    'Black': headers.get('Black', 'Unknown'),
                    'Result': headers.get('Result', '*'),
                    'WhiteElo': headers.get('WhiteElo', ''),
                    'BlackElo': headers.get('BlackElo', ''),
                    'Opening': headers.get('Opening', headers.get('ECO', 'Unknown')),
                    'TimeControl': headers.get('TimeControl', 'Unknown'),
                    'Date': headers.get('Date', ''),
                    'MoveCount': full_move_count
                })
            except Exception as e:
                print(f"Error processing game {i}: {e}")
                continue
    
    df = pd.DataFrame(games)
    
    # Clean data
    df['WhiteElo'] = pd.to_numeric(df['WhiteElo'], errors='coerce')
    df['BlackElo'] = pd.to_numeric(df['BlackElo'], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    print(f"✅ Processed {len(df)} games from PGN file")
    return df

# Load data
try:
    df = pd.read_csv('chess_games_raw.csv')
    print(f"✅ Loaded {len(df)} games from CSV")
except FileNotFoundError:
    print("❌ CSV not found. Please provide data file")
    # Uncomment to load from PGN:
    # df = process_pgn_to_dataframe('your_games.pgn')

# %% [markdown]
# ## 3. Data Preparation

# %%
def prepare_chess_data(df):
    """Clean and prepare chess data for analysis."""
    
    # Convert Elo ratings to numeric
    df['WhiteElo'] = pd.to_numeric(df['WhiteElo'], errors='coerce')
    df['BlackElo'] = pd.to_numeric(df['BlackElo'], errors='coerce')
    
    # Handle dates
    if 'Date' not in df.columns or df['Date'].isna().all():
        df['Date'] = pd.date_range(start='2023-01-01', periods=len(df), freq='D')
    else:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Add game number for tracking
    df['GameNumber'] = range(1, len(df) + 1)
    
    # Calculate average Elo
    df['AverageElo'] = (df['WhiteElo'] + df['BlackElo']) / 2
    
    # Standardize opening names
    df['Opening'] = df['Opening'].fillna('Unknown')
    df['Opening'] = df['Opening'].str.split(':').str[0]
    
    print("✅ Data preparation complete")
    print(f"   Total games: {len(df)}")
    print(f"   Date range: {df['Date'].min()} to {df['Date'].max()}")
    
    return df

df = prepare_chess_data(df)

# %% [markdown]
# ## 4. Complete Chess Player Analyzer Class

# %%
class ChessPlayerAnalyzer:
    """Comprehensive chess analysis tool for individual players"""
    
    def __init__(self, df, config=None):
        """Initialize with chess game data and configuration"""
        self.df = df
        self.config = config
        
    def get_available_players(self, min_games=10):
        """Get list of players with sufficient games"""
        white_counts = self.df['White'].value_counts()
        black_counts = self.df['Black'].value_counts()
        
        all_games = pd.concat([white_counts, black_counts], axis=1, keys=['White', 'Black']).fillna(0)
        all_games['Total'] = all_games['White'] + all_games['Black']
        
        qualified_players = all_games[all_games['Total'] >= min_games]
        return qualified_players.sort_values('Total', ascending=False)
    
    def analyze_player(self, player_name):
        """Complete analysis for a specific player."""
        # Get all games for this player
        player_games = self.df[
            (self.df['White'] == player_name) | 
            (self.df['Black'] == player_name)
        ].copy()
        
        if len(player_games) == 0:
            return {"error": f"No games found for player: {player_name}"}
        
        # Add player-specific columns
        player_games['PlayerColor'] = player_games.apply(
            lambda x: 'White' if x['White'] == player_name else 'Black', axis=1
        )
        
        player_games['PlayerResult'] = player_games.apply(
            lambda x: self._get_player_result(x, player_name), axis=1
        )
        
        player_games['PlayerElo'] = player_games.apply(
            lambda x: x['WhiteElo'] if x['White'] == player_name else x['BlackElo'], axis=1
        )
        
        player_games['OpponentElo'] = player_games.apply(
            lambda x: x['BlackElo'] if x['White'] == player_name else x['WhiteElo'], axis=1
        )
        
        player_games['Opponent'] = player_games.apply(
            lambda x: x['Black'] if x['White'] == player_name else x['White'], axis=1
        )
        
        # Calculate comprehensive statistics
        analysis = {
            'player_name': player_name,
            'games_data': player_games,
            'overall_stats': self._calculate_overall_stats(player_games),
            'color_performance': self._analyze_by_color(player_games),
            'opening_analysis': self._analyze_openings(player_games),
            'rating_progression': self._track_rating_progression(player_games),
            'time_control_stats': self._analyze_by_time_control(player_games),
            'opponent_strength': self._analyze_opponent_strength(player_games),
            'improvement_metrics': self._calculate_improvement(player_games),
            'recent_form': self._analyze_recent_form(player_games),
            'game_duration': self._analyze_game_duration(player_games),
            'duration_patterns': self._analyze_duration_patterns(player_games)
        }
        
        return analysis
    
    def _get_player_result(self, row, player_name):
        """Determine result from player's perspective."""
        if row['White'] == player_name:
            if row['Result'] == '1-0':
                return 'Win'
            elif row['Result'] == '0-1':
                return 'Loss'
            else:
                return 'Draw'
        else:  # Player is Black
            if row['Result'] == '0-1':
                return 'Win'
            elif row['Result'] == '1-0':
                return 'Loss'
            else:
                return 'Draw'
    
    def _calculate_overall_stats(self, player_games):
        """Calculate overall performance statistics."""
        results = player_games['PlayerResult'].value_counts()
        total = len(player_games)
        
        wins = results.get('Win', 0)
        draws = results.get('Draw', 0)
        losses = results.get('Loss', 0)
        
        stats = {
            'total_games': total,
            'wins': wins,
            'draws': draws,
            'losses': losses,
            'win_rate': (wins / total * 100) if total > 0 else 0,
            'draw_rate': (draws / total * 100) if total > 0 else 0,
            'loss_rate': (losses / total * 100) if total > 0 else 0,
            'points_percentage': ((wins + 0.5 * draws) / total * 100) if total > 0 else 0,
            'average_game_length': player_games['MoveCount'].mean() if 'MoveCount' in player_games.columns else None,
            'current_rating': player_games['PlayerElo'].iloc[-1] if not player_games['PlayerElo'].isna().all() else None
        }
        
        return stats
    
    def _analyze_by_color(self, player_games):
        """Analyze performance by piece color."""
        color_stats = {}
        
        for color in ['White', 'Black']:
            color_games = player_games[player_games['PlayerColor'] == color]
            if len(color_games) > 0:
                results = color_games['PlayerResult'].value_counts()
                total = len(color_games)
                
                color_stats[color] = {
                    'games': total,
                    'wins': results.get('Win', 0),
                    'draws': results.get('Draw', 0),
                    'losses': results.get('Loss', 0),
                    'win_rate': (results.get('Win', 0) / total * 100) if total > 0 else 0
                }
        
        return color_stats
    
    def _analyze_openings(self, player_games):
        """Analyze performance by opening."""
        opening_stats = []
        
        for opening in player_games['Opening'].value_counts().head(20).index:
            opening_games = player_games[player_games['Opening'] == opening]
            results = opening_games['PlayerResult'].value_counts()
            total = len(opening_games)
            
            opening_stats.append({
                'opening': opening,
                'games': total,
                'wins': results.get('Win', 0),
                'win_rate': (results.get('Win', 0) / total * 100) if total > 0 else 0
            })
        
        return pd.DataFrame(opening_stats)
    
    def _track_rating_progression(self, player_games):
        """Track rating changes over actual time."""
        if player_games['PlayerElo'].isna().all():
            return pd.DataFrame()
        
        # Ensure Date column exists and is datetime
        if 'Date' not in player_games.columns:
            player_games['Date'] = pd.date_range(start='2023-01-01', periods=len(player_games), freq='D')
        
        player_games['Date'] = pd.to_datetime(player_games['Date'], errors='coerce')
        
        rating_data = player_games[['Date', 'PlayerElo', 'PlayerResult']].copy()
        rating_data = rating_data.dropna(subset=['PlayerElo'])
        
        # Sort by date for chronological order
        rating_data = rating_data.sort_values('Date')
        
        # Add game number for reference only
        rating_data['GameNumber'] = range(1, len(rating_data) + 1)
        
        if len(rating_data) > 10:
            rating_data['RollingAvg'] = rating_data['PlayerElo'].rolling(window=10, min_periods=1).mean()
        
        return rating_data
    
    def _analyze_by_time_control(self, player_games):
        """Analyze performance by time control."""
        if 'TimeControl' not in player_games.columns:
            return {}
        
        time_stats = {}
        for tc in player_games['TimeControl'].unique():
            tc_games = player_games[player_games['TimeControl'] == tc]
            results = tc_games['PlayerResult'].value_counts()
            total = len(tc_games)
            
            if total > 0:
                time_stats[tc] = {
                    'games': total,
                    'win_rate': (results.get('Win', 0) / total * 100)
                }
        
        return time_stats
    
    def _analyze_opponent_strength(self, player_games):
        """Analyze performance vs different strength opponents."""
        rated_games = player_games.dropna(subset=['OpponentElo'])
        
        if len(rated_games) == 0:
            return {}
        
        bins = [-np.inf, 1200, 1400, 1600, 1800, 2000, np.inf]
        labels = ['<1200', '1200-1400', '1400-1600', '1600-1800', '1800-2000', '2000+']
        rated_games['OpponentStrength'] = pd.cut(rated_games['OpponentElo'], bins=bins, labels=labels)
        
        strength_stats = {}
        for strength in labels:
            strength_games = rated_games[rated_games['OpponentStrength'] == strength]
            if len(strength_games) > 0:
                results = strength_games['PlayerResult'].value_counts()
                total = len(strength_games)
                strength_stats[strength] = {
                    'games': total,
                    'win_rate': (results.get('Win', 0) / total * 100)
                }
        
        return strength_stats
    
    def _calculate_improvement(self, player_games):
        """Calculate improvement metrics."""
        if len(player_games) < 20:
            return {"message": "Not enough games for trend analysis"}
        
        q_size = len(player_games) // 4
        quarters = {
            'Q1': player_games.iloc[:q_size],
            'Q2': player_games.iloc[q_size:2*q_size],
            'Q3': player_games.iloc[2*q_size:3*q_size],
            'Q4': player_games.iloc[3*q_size:]
        }
        
        improvement = {}
        for q_name, q_games in quarters.items():
            results = q_games['PlayerResult'].value_counts()
            total = len(q_games)
            improvement[q_name] = {
                'win_rate': (results.get('Win', 0) / total * 100) if total > 0 else 0,
                'games': total
            }
        
        win_rates = [improvement[q]['win_rate'] for q in ['Q1', 'Q2', 'Q3', 'Q4']]
        improvement['trend'] = 'improving' if win_rates[-1] > win_rates[0] else 'declining'
        improvement['change'] = win_rates[-1] - win_rates[0]
        
        return improvement
    
    def _analyze_recent_form(self, player_games, last_n=20):
        """Analyze recent game performance."""
        recent_games = player_games.tail(last_n)
        
        if len(recent_games) == 0:
            return {}
        
        results = recent_games['PlayerResult'].value_counts()
        total = len(recent_games)
        
        return {
            'games': total,
            'wins': results.get('Win', 0),
            'win_rate': (results.get('Win', 0) / total * 100) if total > 0 else 0,
            'streak': self._calculate_streak(recent_games)
        }
    
    def _calculate_streak(self, games):
        """Calculate current winning/losing streak."""
        if len(games) == 0:
            return {'type': 'none', 'count': 0}
        
        last_results = games['PlayerResult'].tail(10).tolist()
        if not last_results:
            return {'type': 'none', 'count': 0}
        
        current_result = last_results[-1]
        count = 1
        
        for result in reversed(last_results[:-1]):
            if result == current_result:
                count += 1
            else:
                break
        
        return {'type': current_result.lower(), 'count': count}
    
    def _analyze_game_duration(self, player_games):
        """Analyze game duration patterns and statistics."""
        if 'MoveCount' not in player_games.columns or player_games['MoveCount'].isna().all():
            return {"message": "No move count data available"}
        
        # Basic statistics
        duration_stats = {
            'mean': player_games['MoveCount'].mean(),
            'median': player_games['MoveCount'].median(),
            'std': player_games['MoveCount'].std(),
            'min': player_games['MoveCount'].min(),
            'max': player_games['MoveCount'].max(),
            'q25': player_games['MoveCount'].quantile(0.25),
            'q75': player_games['MoveCount'].quantile(0.75),
            'total_games_with_data': player_games['MoveCount'].notna().sum()
        }
        
        # Duration by result
        duration_by_result = {}
        for result in ['Win', 'Loss', 'Draw']:
            result_games = player_games[player_games['PlayerResult'] == result]
            if not result_games.empty and 'MoveCount' in result_games.columns:
                duration_by_result[result] = {
                    'mean': result_games['MoveCount'].mean(),
                    'median': result_games['MoveCount'].median(),
                    'count': len(result_games)
                }
        
        # Duration by time control
        duration_by_time_control = {}
        if 'TimeControl' in player_games.columns:
            for tc in player_games['TimeControl'].unique():
                tc_games = player_games[player_games['TimeControl'] == tc]
                if not tc_games.empty and not tc_games['MoveCount'].isna().all():
                    duration_by_time_control[tc] = {
                        'mean': tc_games['MoveCount'].mean(),
                        'median': tc_games['MoveCount'].median(),
                        'count': len(tc_games)
                    }
        
        # Categorize game lengths
        bins = [0, 20, 40, 60, 80, 100, float('inf')]
        labels = ['Very Short (<20)', 'Short (20-40)', 'Medium (40-60)', 
                  'Long (60-80)', 'Very Long (80-100)', 'Marathon (100+)']
        player_games['GameLengthCategory'] = pd.cut(
            player_games['MoveCount'], bins=bins, labels=labels
        )
        
        length_distribution = player_games['GameLengthCategory'].value_counts().to_dict()
        
        # Create detailed data for visualization
        detailed_data = player_games[['Date', 'MoveCount', 'PlayerResult', 'GameLengthCategory']].copy()
        if len(detailed_data) > 10:
            detailed_data['MoveCountRolling'] = detailed_data['MoveCount'].rolling(
                window=10, min_periods=1
            ).mean()
        
        return {
            'statistics': duration_stats,
            'by_result': duration_by_result,
            'by_time_control': duration_by_time_control,
            'length_categories': length_distribution,
            'detailed_data': detailed_data
        }
    
    def _analyze_duration_patterns(self, player_games):
        """Analyze patterns in game duration for strategic insights."""
        if 'MoveCount' not in player_games.columns:
            return {}
        
        patterns = {}
        
        # Quick wins vs grinding victories
        wins = player_games[player_games['PlayerResult'] == 'Win']
        if not wins.empty:
            quick_wins = wins[wins['MoveCount'] <= 30]
            long_wins = wins[wins['MoveCount'] >= 60]
            patterns['quick_win_rate'] = (len(quick_wins) / len(wins) * 100) if len(wins) > 0 else 0
            patterns['long_win_rate'] = (len(long_wins) / len(wins) * 100) if len(wins) > 0 else 0
        
        # Resignation patterns
        losses = player_games[player_games['PlayerResult'] == 'Loss']
        if not losses.empty:
            early_losses = losses[losses['MoveCount'] <= 25]
            patterns['early_resignation_rate'] = (len(early_losses) / len(losses) * 100) if len(losses) > 0 else 0
        
        # Endgame frequency
        total_games = len(player_games[player_games['MoveCount'].notna()])
        endgame_games = len(player_games[player_games['MoveCount'] > 60])
        patterns['endgame_frequency'] = (endgame_games / total_games * 100) if total_games > 0 else 0
        
        return patterns

# %% [markdown]
# ## 5. Player Selection and Analysis

# %%
def select_player_interactive(analyzer):
    """Interactive player selection with validation"""
    players = analyzer.get_available_players()
    
    if players.empty:
        print(f"❌ No players found with at least 10 games")
        return None
    
    print("\n🎯 Available Players for Analysis:")
    print("=" * 60)
    
    display_count = min(20, len(players))
    for i, (player, data) in enumerate(players.head(display_count).iterrows(), 1):
        print(f"{i:2}. {player:30} - {int(data['Total']):4} games")
    
    print("\n" + "=" * 60)
    
    selected_player = players.index[0]
    print(f"\n✅ Selected player: {selected_player}")
    
    return selected_player

# Initialize analyzer
analyzer = ChessPlayerAnalyzer(df)

# Select player
PLAYER_NAME = select_player_interactive(analyzer)

if PLAYER_NAME:
    # Run analysis
    analysis = analyzer.analyze_player(PLAYER_NAME)
    
    if 'error' not in analysis:
        # Display results
        stats = analysis['overall_stats']
        print(f"\n📊 Performance Summary for {PLAYER_NAME}")
        print("=" * 60)
        print(f"Total Games: {stats['total_games']}")
        print(f"Win Rate: {stats['win_rate']:.1f}%")
        print(f"Points %: {stats['points_percentage']:.1f}%")
        print(f"Average Game Length: {stats['average_game_length']:.1f} moves")

# %% [markdown]
# ## 6. Generate Comprehensive Report with Export Options

# %%
def generate_player_report(analyzer, player_name, save_html=True, save_images=True, save_formats=['png']):
    """Generate comprehensive report for a player with multiple export options"""
    analysis = analyzer.analyze_player(player_name)
    
    if 'error' in analysis:
        print(f"❌ {analysis['error']}")
        return None
    
    stats = analysis['overall_stats']
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Win/Loss Distribution',
            'Rating Progression Over Time',
            'Top 10 Openings',
            'Performance by Color',
            'Performance vs Opponent Strength',
            'Improvement Over Time'
        ),
        specs=[
            [{'type': 'pie'}, {'type': 'scatter'}],
            [{'type': 'bar'}, {'type': 'bar'}],
            [{'type': 'bar'}, {'type': 'bar'}]
        ],
        vertical_spacing=0.1,
        horizontal_spacing=0.12
    )
    
    # 1. Win/Loss Pie Chart
    fig.add_trace(
        go.Pie(
            labels=['Wins', 'Draws', 'Losses'],
            values=[stats['wins'], stats['draws'], stats['losses']],
            marker_colors=['#2ecc71', '#95a5a6', '#e74c3c'],
            hole=0.3
        ),
        row=1, col=1
    )
    
    # 2. Rating Progression Over Time
    rating_data = analysis['rating_progression']
    if not rating_data.empty and 'Date' in rating_data.columns:
        fig.add_trace(
            go.Scatter(
                x=rating_data['Date'],
                y=rating_data['PlayerElo'],
                mode='lines+markers',
                name='Rating',
                line=dict(color='#3498db', width=2),
                hovertemplate='Date: %{x|%Y-%m-%d}<br>Rating: %{y}<extra></extra>'
            ),
            row=1, col=2
        )
        
        if 'RollingAvg' in rating_data.columns:
            fig.add_trace(
                go.Scatter(
                    x=rating_data['Date'],
                    y=rating_data['RollingAvg'],
                    mode='lines',
                    name='10-Game Avg',
                    line=dict(color='#e74c3c', width=2, dash='dash'),
                    hovertemplate='Date: %{x|%Y-%m-%d}<br>Avg: %{y}<extra></extra>'
                ),
                row=1, col=2
            )
    
    # 3. Top Openings
    opening_df = analysis['opening_analysis']
    if not opening_df.empty:
        top_openings = opening_df.head(10)
        fig.add_trace(
            go.Bar(
                x=top_openings['opening'],
                y=top_openings['win_rate'],
                text=[f"{wr:.1f}%" for wr in top_openings['win_rate']],
                textposition='outside',
                marker_color='#9b59b6'
            ),
            row=2, col=1
        )
    
    # 4. Performance by Color
    color_data = analysis['color_performance']
    if color_data:
        colors = list(color_data.keys())
        win_rates = [color_data[c]['win_rate'] for c in colors]
        games = [color_data[c]['games'] for c in colors]
        
        fig.add_trace(
            go.Bar(
                x=colors,
                y=win_rates,
                text=[f"{wr:.1f}%<br>{g} games" for wr, g in zip(win_rates, games)],
                textposition='outside',
                marker_color=['#ecf0f1', '#34495e']
            ),
            row=2, col=2
        )
    
    # 5. Performance vs Opponent Strength
    strength_data = analysis['opponent_strength']
    if strength_data:
        strengths = list(strength_data.keys())
        win_rates = [strength_data[s]['win_rate'] for s in strengths]
        
        fig.add_trace(
            go.Bar(
                x=strengths,
                y=win_rates,
                text=[f"{wr:.1f}%" for wr in win_rates],
                textposition='outside',
                marker_color='#f39c12'
            ),
            row=3, col=1
        )
    
    # 6. Improvement Over Time
    improvement = analysis['improvement_metrics']
    if isinstance(improvement, dict) and 'Q1' in improvement:
        quarters = ['Q1', 'Q2', 'Q3', 'Q4']
        win_rates = [improvement[q]['win_rate'] for q in quarters]
        
        colors_list = ['#e74c3c' if wr < win_rates[0] else '#2ecc71' for wr in win_rates]
        
        fig.add_trace(
            go.Bar(
                x=quarters,
                y=win_rates,
                text=[f"{wr:.1f}%" for wr in win_rates],
                textposition='outside',
                marker_color=colors_list
            ),
            row=3, col=2
        )
    
    # Update layout
    fig.update_layout(
        title=f"<b>Chess Performance Analysis: {player_name}</b><br>" +
              f"<sub>Total: {stats['total_games']} games | " +
              f"Win Rate: {stats['win_rate']:.1f}% | " +
              f"Points: {stats['points_percentage']:.1f}%</sub>",
        height=1200,
        showlegend=False,
        template='plotly_white'
    )
    
    # Update axes
    fig.update_xaxes(title="Date", row=1, col=2)
    fig.update_yaxes(title="Rating", row=1, col=2)
    fig.update_xaxes(title="Opening", row=2, col=1, tickangle=45)
    fig.update_yaxes(title="Win Rate (%)", row=2, col=1)
    fig.update_xaxes(title="Color", row=2, col=2)
    fig.update_yaxes(title="Win Rate (%)", row=2, col=2)
    fig.update_xaxes(title="Opponent Rating", row=3, col=1)
    fig.update_yaxes(title="Win Rate (%)", row=3, col=1)
    fig.update_xaxes(title="Quarter", row=3, col=2)
    fig.update_yaxes(title="Win Rate (%)", row=3, col=2)
    
    # Create output directory
    output_dir = 'reports'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    player_filename = player_name.replace(' ', '_')
    
    # Save HTML
    if save_html:
        output_path = os.path.join(output_dir, f"{player_filename}_report.html")
        fig.write_html(output_path)
        print(f"✅ HTML report saved to: {output_path}")
    
    # Save static images (requires kaleido)
    if save_images:
        for format in save_formats:
            try:
                output_path = os.path.join(output_dir, f"{player_filename}_report.{format}")
                if format == 'png':
                    fig.write_image(output_path, width=1920, height=1200, scale=2)
                else:
                    fig.write_image(output_path)
                print(f"✅ {format.upper()} saved to: {output_path}")
            except Exception as e:
                print(f"⚠️ Could not save {format}: Install kaleido for static exports")
    
    return fig

# %% [markdown]
# ## 7. Game Duration Analysis and Visualization

# %%
def visualize_game_duration(analysis, save_plot=True, player_name="Player"):
    """Create comprehensive visualization of game duration analysis."""
    duration_data = analysis.get('game_duration', {})
    
    if isinstance(duration_data, dict) and 'message' in duration_data:
        print(f"❌ {duration_data['message']}")
        return None
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=(
            'Move Count Distribution',
            'Game Length by Result',
            'Duration Trend Over Time',
            'Game Length Categories',
            'Duration by Time Control',
            'Duration Statistics'
        ),
        specs=[
            [{'type': 'histogram'}, {'type': 'box'}, {'type': 'scatter'}],
            [{'type': 'bar'}, {'type': 'bar'}, {'type': 'table'}]
        ],
        vertical_spacing=0.15,
        horizontal_spacing=0.12
    )
    
    games_data = analysis['games_data']
    
    # 1. Histogram of move counts
    if 'MoveCount' in games_data.columns:
        fig.add_trace(
            go.Histogram(
                x=games_data['MoveCount'].dropna(),
                nbinsx=30,
                marker_color='#3498db',
                name='Move Count'
            ),
            row=1, col=1
        )
    
    # 2. Box plot by result
    if 'PlayerResult' in games_data.columns and 'MoveCount' in games_data.columns:
        for result in ['Win', 'Loss', 'Draw']:
            result_data = games_data[games_data['PlayerResult'] == result]['MoveCount'].dropna()
            if not result_data.empty:
                fig.add_trace(
                    go.Box(
                        y=result_data,
                        name=result,
                        marker_color={'Win': '#2ecc71', 'Loss': '#e74c3c', 'Draw': '#95a5a6'}[result]
                    ),
                    row=1, col=2
                )
    
    # 3. Duration trend over time
    detailed_data = duration_data.get('detailed_data', pd.DataFrame())
    if not detailed_data.empty and 'Date' in detailed_data.columns:
        fig.add_trace(
            go.Scatter(
                x=detailed_data['Date'],
                y=detailed_data['MoveCount'],
                mode='markers',
                marker=dict(size=4, color='#3498db', opacity=0.5),
                name='Individual Games'
            ),
            row=1, col=3
        )
        
        if 'MoveCountRolling' in detailed_data.columns:
            fig.add_trace(
                go.Scatter(
                    x=detailed_data['Date'],
                    y=detailed_data['MoveCountRolling'],
                    mode='lines',
                    line=dict(color='#e74c3c', width=2),
                    name='10-Game Average'
                ),
                row=1, col=3
            )
    
    # 4. Game length categories
    length_cats = duration_data.get('length_categories', {})
    if length_cats:
        categories = list(length_cats.keys())
        counts = list(length_cats.values())
        
        fig.add_trace(
            go.Bar(
                x=categories,
                y=counts,
                text=counts,
                textposition='outside',
                marker_color='#9b59b6'
            ),
            row=2, col=1
        )
    
    # 5. Duration by time control
    tc_data = duration_data.get('by_time_control', {})
    if tc_data:
        time_controls = list(tc_data.keys())[:5]
        avg_moves = [tc_data[tc]['mean'] for tc in time_controls]
        
        fig.add_trace(
            go.Bar(
                x=time_controls,
                y=avg_moves,
                text=[f"{m:.1f}" for m in avg_moves],
                textposition='outside',
                marker_color='#f39c12'
            ),
            row=2, col=2
        )
    
    # 6. Statistics table
    stats = duration_data.get('statistics', {})
    if stats:
        table_data = [
            ['Mean', f"{stats.get('mean', 0):.1f}"],
            ['Median', f"{stats.get('median', 0):.1f}"],
            ['Std Dev', f"{stats.get('std', 0):.1f}"],
            ['Min', f"{stats.get('min', 0):.0f}"],
            ['Max', f"{stats.get('max', 0):.0f}"],
            ['Q25', f"{stats.get('q25', 0):.1f}"],
            ['Q75', f"{stats.get('q75', 0):.1f}"]
        ]
        
        fig.add_trace(
            go.Table(
                header=dict(values=['Metric', 'Value'],
                           fill_color='#3498db',
                           font=dict(color='white')),
                cells=dict(values=[[row[0] for row in table_data],
                                  [row[1] for row in table_data]],
                          fill_color='#ecf0f1')
            ),
            row=2, col=3
        )
    
    # Update layout
    fig.update_layout(
        title=f"<b>Game Duration Analysis: {player_name}</b>",
        height=800,
        showlegend=False,
        template='plotly_white'
    )
    
    # Update axes
    fig.update_xaxes(title="Move Count", row=1, col=1)
    fig.update_yaxes(title="Frequency", row=1, col=1)
    fig.update_yaxes(title="Move Count", row=1, col=2)
    fig.update_xaxes(title="Date", row=1, col=3)
    fig.update_yaxes(title="Move Count", row=1, col=3)
    fig.update_xaxes(title="Category", row=2, col=1, tickangle=45)
    fig.update_yaxes(title="Count", row=2, col=1)
    fig.update_xaxes(title="Time Control", row=2, col=2, tickangle=45)
    fig.update_yaxes(title="Avg Moves", row=2, col=2)
    
    if save_plot:
        output_dir = 'reports'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        output_path = os.path.join(output_dir, f"{player_name.replace(' ', '_')}_duration_analysis.html")
        fig.write_html(output_path)
        print(f"✅ Duration analysis saved to: {output_path}")
    
    return fig

def print_duration_insights(analysis, player_name="Player"):
    """Print key insights from game duration analysis."""
    duration_data = analysis.get('game_duration', {})
    patterns = analysis.get('duration_patterns', {})
    
    if isinstance(duration_data, dict) and 'message' in duration_data:
        print(f"❌ {duration_data['message']}")
        return
    
    stats = duration_data.get('statistics', {})
    
    print(f"\n📊 Game Duration Analysis for {player_name}")
    print("=" * 60)
    
    print(f"\n📈 Duration Statistics:")
    print(f"  Average game length: {stats.get('mean', 0):.1f} moves")
    print(f"  Median game length: {stats.get('median', 0):.1f} moves")
    print(f"  Shortest game: {stats.get('min', 0):.0f} moves")
    print(f"  Longest game: {stats.get('max', 0):.0f} moves")
    print(f"  Standard deviation: {stats.get('std', 0):.1f} moves")
    
    by_result = duration_data.get('by_result', {})
    if by_result:
        print(f"\n🎯 Average Duration by Result:")
        for result, data in by_result.items():
            print(f"  {result}: {data['mean']:.1f} moves (median: {data['median']:.1f})")
    
    if patterns:
        print(f"\n🎮 Strategic Patterns:")
        if 'quick_win_rate' in patterns:
            print(f"  Quick wins (<30 moves): {patterns['quick_win_rate']:.1f}% of wins")
        if 'long_win_rate' in patterns:
            print(f"  Long wins (>60 moves): {patterns['long_win_rate']:.1f}% of wins")
        if 'early_resignation_rate' in patterns:
            print(f"  Early resignations (<25 moves): {patterns['early_resignation_rate']:.1f}% of losses")
        if 'endgame_frequency' in patterns:
            print(f"  Endgame frequency (>60 moves): {patterns['endgame_frequency']:.1f}% of games")
    
    print("\n" + "=" * 60)

# %% [markdown]
# ## 8. Run Complete Analysis

# %%
# Generate comprehensive report
if PLAYER_NAME:
    print(f"\n📊 Generating complete report for {PLAYER_NAME}...")
    
    # Main performance report
    fig = generate_player_report(analyzer, PLAYER_NAME, save_html=True, save_images=False)
    if fig:
        fig.show()
    
    # Duration analysis
    print_duration_insights(analysis, PLAYER_NAME)
    duration_fig = visualize_game_duration(analysis, save_plot=True, player_name=PLAYER_NAME)
    if duration_fig:
        duration_fig.show()

# %% [markdown]
# ## 9. Batch Analysis Function

# %%
def batch_analyze_players(analyzer, player_list=None, top_n=5):
    """Batch analyze multiple players"""
    if player_list is None:
        available = analyzer.get_available_players()
        player_list = available.head(top_n).index.tolist()
    
    print(f"\n📊 Batch Analysis for {len(player_list)} players")
    print("=" * 60)
    
    results = []
    for i, player in enumerate(player_list, 1):
        try:
            print(f"\n{i}. Analyzing {player}...")
            
            # Generate reports
            fig = generate_player_report(analyzer, player, save_html=True, save_images=False)
            duration_fig = visualize_game_duration(
                analyzer.analyze_player(player), 
                save_plot=True, 
                player_name=player
            )
            
            # Collect summary
            analysis = analyzer.analyze_player(player)
            if 'error' not in analysis:
                stats = analysis['overall_stats']
                duration_stats = analysis['game_duration']['statistics']
                results.append({
                    'Player': player,
                    'Games': stats['total_games'],
                    'Win_Rate': stats['win_rate'],
                    'Avg_Game_Length': duration_stats['mean']
                })
                print(f"   ✅ Success - Win Rate: {stats['win_rate']:.1f}%")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    if results:
        summary_df = pd.DataFrame(results)
        summary_df.to_csv('reports/batch_summary.csv', index=False)
        print(f"\n✅ Batch analysis complete. Summary saved.")
        print(summary_df)
        
        return summary_df
    
    return None

# Uncomment to run batch analysis:
# batch_results = batch_analyze_players(analyzer, top_n=3)

✅ Libraries imported successfully
✅ Loaded 4999 games from CSV
✅ Data preparation complete
   Total games: 4999
   Date range: 2023-01-01 00:00:00 to 2036-09-07 00:00:00

🎯 Available Players for Analysis:
 1. rick                           -  111 games
 2. Yannyi                         -   84 games
 3. fil77                          -   59 games
 4. vino                           -   59 games
 5. BrassKnight                    -   58 games
 6. Murali                         -   55 games
 7. Beijamim                       -   54 games
 8. oilmanesh                      -   53 games
 9. bmv                            -   51 games
10. controlaltdelete               -   51 games
11. Machaquito                     -   50 games
12. ribarisah                      -   49 games
13. el_panuelo                     -   48 games
14. trom                           -   48 games
15. smit                           -   48 games
16. SaltaCavallo                   -   47 games
17. benedicto16            


📊 Game Duration Analysis for rick

📈 Duration Statistics:
  Average game length: 57.1 moves
  Median game length: 54.0 moves
  Shortest game: 3 moves
  Longest game: 143 moves
  Standard deviation: 32.3 moves

🎯 Average Duration by Result:
  Win: 48.7 moves (median: 46.0)
  Loss: 63.6 moves (median: 61.0)
  Draw: 118.5 moves (median: 118.5)

🎮 Strategic Patterns:
  Quick wins (<30 moves): 30.4% of wins
  Long wins (>60 moves): 33.9% of wins
  Early resignations (<25 moves): 11.3% of losses
  Endgame frequency (>60 moves): 40.5% of games

✅ Duration analysis saved to: reports/rick_duration_analysis.html
